# The JOIN operation

![rel](https://sqlzoo.net/w/images/c/ce/FootballERD.png)

## JOIN and UEFA EURO 2012

This tutorial introduces `JOIN` which allows you to use data from two or more tables. The tables contain all matches and goals from UEFA EURO 2012 Football Championship in Poland and Ukraine.

The data is available (mysql format) at <http://sqlzoo.net/euro2012.sql>

In [1]:
library(tidyverse)
library(DBI)
library(getPass)
drv <- switch(Sys.info()['sysname'],
             Windows="PostgreSQL Unicode(x64)",
             Darwin="/usr/local/lib/psqlodbcw.so",
             Linux="PostgreSQL")
con <- dbConnect(
  odbc::odbc(),
  driver = drv,
  Server = "localhost",
  Database = "sqlzoo",
  UID = "postgres",
  PWD = getPass("Password?"),
  Port = 5432
)
options(repr.matrix.max.rows=20)

-- Attaching packages --------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.0     v purrr   0.3.4
v tibble  3.0.1     v dplyr   0.8.5
v tidyr   1.0.2     v stringr 1.4.0
v readr   1.3.1     v forcats 0.5.0

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



Password? ·········


## 1.
The first example shows the goal scored by a player with the last name 'Bender'. The `*` says to list all the columns in the table - a shorter way of saying `matchid, teamid, player, gtime`

**Modify it to show the matchid and player name for all goals scored by Germany. To identify German players, check for: `teamid = 'GER'`**

In [2]:
game <- dbReadTable(con, 'game')
goal <- dbReadTable(con, 'goal')
eteam <- dbReadTable(con, 'eteam')

In [3]:
goal %>% 
    filter(teamid=='GER') %>% 
    select(matchid, player)

matchid,player
<int>,<chr>
1008,Mario Gómez
1010,Mario Gómez
1010,Mario Gómez
1012,Lukas Podolski
1012,Lars Bender
1026,Philipp Lahm
1026,Sami Khedira
1026,Miroslav Klose
1026,Marco Reus


## 2.

From the previous query you can see that Lars Bender's scored a goal in game 1012. Now we want to know what teams were playing in that match.

Notice in the that the column `matchid `in the `goal` table corresponds to the `id` column in the `game` table. We can look up information about game 1012 by finding that row in the `game` table.

**Show id, stadium, team1, team2 for just game 1012**

In [4]:
game %>% 
    filter(id==1012) %>%
    select(id, stadium, team1, team2)

id,stadium,team1,team2
<int>,<chr>,<chr>,<chr>
1012,Arena Lviv,DEN,GER


## 3.
You can combine the two steps into a single query with a JOIN.

```sql
SELECT *
  FROM game JOIN goal ON (id=matchid)
```

The **FROM** clause says to merge data from the goal table with that from the game table. The **ON** says how to figure out which rows in **game** go with which rows in **goal** - the **matchid** from **goal** must match **id** from **game**. (If we wanted to be more clear/specific we could say
`ON (game.id=goal.matchid)`

The code below shows the player (from the goal) and stadium name (from the game table) for every goal scored.

**Modify it to show the player, teamid, stadium and mdate for every German goal.**

In [5]:
game %>% 
    inner_join(goal, by=c(id="matchid")) %>% 
    filter(teamid=='GER') %>%
    select(player, teamid, stadium, mdate)

player,teamid,stadium,mdate
<chr>,<chr>,<chr>,<chr>
Mario Gómez,GER,Arena Lviv,9 June 2012
Mario Gómez,GER,Metalist Stadium,13 June 2012
Mario Gómez,GER,Metalist Stadium,13 June 2012
Lukas Podolski,GER,Arena Lviv,17 June 2012
Lars Bender,GER,Arena Lviv,17 June 2012
Philipp Lahm,GER,PGE Arena Gdansk,22 June 2012
Sami Khedira,GER,PGE Arena Gdansk,22 June 2012
Miroslav Klose,GER,PGE Arena Gdansk,22 June 2012
Marco Reus,GER,PGE Arena Gdansk,22 June 2012


## 4.
Use the same `JOIN` as in the previous question.

**Show the team1, team2 and player for every goal scored by a player called Mario `player LIKE 'Mario%'`**

In [6]:
game %>%
    inner_join(goal, by=c(id='matchid')) %>% 
    filter(str_starts(player, 'Mario')) %>%
    select(team1, team2, player)

team1,team2,player
<chr>,<chr>,<chr>
GER,POR,Mario Gómez
NED,GER,Mario Gómez
NED,GER,Mario Gómez
IRL,CRO,Mario Mandžukic
IRL,CRO,Mario Mandžukic
ITA,CRO,Mario Mandžukic
ITA,IRL,Mario Balotelli
GER,ITA,Mario Balotelli
GER,ITA,Mario Balotelli


## 5.

The table `eteam` gives details of every national team including the coach. You can `JOIN` `goal` to `eteam` using the phrase goal `JOIN eteam on teamid=id`

**Show `player, teamid, coach, gtime` for all goals scored in the first 10 minutes `gtime<=10`**

In [7]:
goal %>% 
    inner_join(eteam, by=c(teamid="id")) %>%
    filter(gtime<=10) %>%
    select(player, teamid, coach, gtime)

player,teamid,coach,gtime
<chr>,<chr>,<chr>,<int>
Petr Jirácek,CZE,Michal Bílek,3
Václav Pilar,CZE,Michal Bílek,6
Mario Mandžukic,CRO,Slaven Bilic,3
Fernando Torres,ESP,Vicente del Bosque,4


## 6.

To `JOIN` `game` with `eteam` you could use either
`game JOIN eteam ON (team1=eteam.id)` or `game JOIN eteam ON (team2=eteam.id)`

Notice that because `id` is a column name in both `game` and `eteam` you must specify `eteam.id` instead of just `id`

**List the the dates of the matches and the name of the team in which 'Fernando Santos' was the team1 coach.**

In [8]:
game %>% 
    inner_join(eteam, by=c(team1="id")) %>%
    filter(coach=='Fernando Santos') %>% 
    select(mdate, teamname)

mdate,teamname
<chr>,<chr>
12 June 2012,Greece
16 June 2012,Greece


## 7.

**List the player for every goal scored in a game where the stadium was 'National Stadium, Warsaw'**

In [9]:
goal %>% 
    inner_join(game, by=c(matchid="id")) %>% 
    filter(stadium=='National Stadium, Warsaw') %>% 
    select(player)

player
<chr>
Robert Lewandowski
Dimitris Salpingidis
Alan Dzagoev
Jakub Blaszczykowski
Giorgos Karagounis
Cristiano Ronaldo
Mario Balotelli
Mario Balotelli
Mesut Özil


## 8. More difficult questions

The example query shows all goals scored in the Germany-Greece quarterfinal.
**Instead show the name of all players who scored a goal against Germany.**

> __HINT__   
> Select goals scored only by non-German players in matches where GER was the id of either **team1** or **team2**.
> You can use `teamid!='GER'` to prevent listing German players.
> You can use `DISTINCT` to stop players being listed twice.

In [10]:
game %>% 
    inner_join(goal, by=c(id="matchid")) %>%
    filter((team1=='GER' | team2=='GER') &
           teamid != 'GER') %>%
    select(player) %>%
    distinct

player
<chr>
Robin van Persie
Michael Krohn-Dehli
Georgios Samaras
Dimitris Salpingidis
Mario Balotelli


## 9.
Show teamname and the total number of goals scored.

> __COUNT and GROUP BY__  
> You should COUNT(*) in the SELECT line and GROUP BY teamname

In [11]:
eteam %>% 
    inner_join(goal, by=c(id="teamid")) %>%
    group_by(teamname) %>% 
    summarise(goals=n())

teamname,goals
<chr>,<int>
Croatia,4
Czech Republic,4
Denmark,4
England,5
France,3
Germany,10
Greece,5
Italy,6
Netherlands,2


## 10.

**Show the stadium and the number of goals scored in each stadium.**

In [12]:
game %>%
    inner_join(goal, by=c(id="matchid")) %>%
    group_by(stadium) %>% 
    summarise(goals=n())

stadium,goals
<chr>,<int>
Arena Lviv,9
Donbass Arena,7
Metalist Stadium,7
"National Stadium, Warsaw",9
Olimpiyskiy National Sports Complex,14
PGE Arena Gdansk,13
Stadion Miejski (Poznan),8
Stadion Miejski (Wroclaw),9


## 11.
**For every match involving 'POL', show the matchid, date and the number of goals scored.**

In [13]:
game %>% 
    inner_join(goal, by=c(id="matchid")) %>%
    filter(team1=='POL' | team2=='POL') %>%
    select(id, mdate) %>% 
    group_by(id, mdate) %>% 
    summarise(goals=n())

id,mdate,goals
<int>,<chr>,<int>
1001,8 June 2012,2
1004,12 June 2012,2
1005,16 June 2012,1


## 12.
**For every match where 'GER' scored, show matchid, match date and the number of goals scored by 'GER'**

In [14]:
game %>% 
    inner_join(goal, by=c(id="matchid")) %>%
    filter(teamid=='GER') %>%
    group_by(id, mdate) %>%
    summarise(goals=n())

id,mdate,goals
<int>,<chr>,<int>
1008,9 June 2012,1
1010,13 June 2012,2
1012,17 June 2012,2
1026,22 June 2012,4
1030,28 June 2012,1


## 13.
List every match with the goals scored by each team as shown. This will use "CASE WHEN" which has not been explained in any previous exercises.

  mdate | team1 | score1 | team2 | score2
--------|-------|--------|-------|----------
1 July 2012 | ESP | 4 | ITA | 0
10 June 2012 | ESP | 1 | ITA | 1
10 June 2012 | IRL | 1 | CRO | 3
... | ... | ... | ... | ... | ...

Notice in the query given every goal is listed. If it was a team1 goal then a 1 appears in score1, otherwise there is a 0. You could SUM this column to get a count of the goals scored by team1. **Sort your result by mdate, matchid, team1 and team2.**

In [15]:
game %>%
    left_join(goal, by=c(id="matchid")) %>% 
    mutate(goal1=if_else(is.na(teamid), FALSE, team1==teamid), 
           goal2=if_else(is.na(teamid), FALSE, team2==teamid)) %>%
    group_by(id, mdate, team1, team2) %>% 
    summarise(score1=sum(goal1),
              score2=sum(goal2)) %>%
    arrange(mdate, id) %>% 
    ungroup() %>%
    select(mdate, team1, score1, team2, score2)

mdate,team1,score1,team2,score2
<chr>,<chr>,<int>,<chr>,<int>
1 July 2012,ESP,4,ITA,0
10 June 2012,ESP,1,ITA,1
10 June 2012,IRL,1,CRO,3
11 June 2012,FRA,1,ENG,1
11 June 2012,UKR,2,SWE,1
12 June 2012,GRE,1,CZE,2
12 June 2012,POL,1,RUS,1
13 June 2012,DEN,2,POR,3
13 June 2012,NED,1,GER,2


In [16]:
dbDisconnect(con)